# Assignment 1
## Digital Musicology

@authors: Joris Monnet, Xingyu Pan, Yutaka Osaki, Yiwei Liu

Due Date: 24/04/2024


In [ ]:
import music21

## Task A

TODO: open and listen a midi file

# Task B

From your analysis in task A, think about how to model these aspects computationally. In this task, choose one or more aspect(s) in your analysis you would like to model.

You should write an algorithm that outputs a MIDI file from an musicXML or the unperformed MIDI. In your report, please clearly describe your model.

In [ ]:
# TODO coding part

## Task C

Compare your generated MIDI with both the unperformed MIDI and the human performace recording. Discuss their differences and potentially what and how you can further improve. This should be part of your discussion section in the report.

In [ ]:
# TODO visualizations of differences ?